# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
from azureml.core import Workspace, Dataset,  ComputeTarget, Experiment
from azureml.core.compute import AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Model
from azureml.core import Environment
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
dataset_name = 'heart-failure-data'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [8]:
# inspect the dataframe 
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## 
Compute Cluster Configuration

In [5]:
ws = Workspace.from_config()


experiment_name = 'capstone-project-hyper'
experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()
cluster_name = "training-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 6,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,min_node_count=None,timeout_in_minutes=10,
    )


print(f'compute target: {compute_target.get_status().serialize()}')

Creating a new compute target (error: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name training-cluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name training-cluster not found in provided workspace"
    }
}
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
compute target: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-11-03T16:32:00.557000+00:00', 'errors': None, 'creationTime': '2024-11-03T16:31:51.981228+00:00', 'modifiedTime': '2024-11-03T16:32:02.008627+00:00', 'provisioningState': 'Succ

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

RandomParameterSampling - This is a strategy used to randomly sample hyperparameters for model training. During the experiment, the ML framework will randomly select values from the specified choices for each hyperparameter.

learning_rate: The rate at which the model learns during training. The choices given are 0.01, 0.05, 0.1, 0.25, 1.0. n_estimators: The number of trees in an ensemble model (like a Random Forest or Gradient Boosting). The choices are 1, 5, 10, 25.

BanditPolicy = This is an early stopping policy that helps in terminating poorly performing runs early, based on performance compared to the best run.

evaluation_interval: Specifies how often (in terms of number of iterations) the policy should evaluate the performance of the runs. Here, it is set to 3, meaning the performance will be checked every 3 iterations. slack_factor: This sets the tolerance level for how much worse a run can perform relative to the best run before being terminated. A slack_factor of 0.2 means that a run can perform up to 20% worse than the best performing run before it is stopped.

In this project we used Gradient boosting as it builds an ensemble of trees sequentially, where each tree tries to correct the errors of the previous one. This iterative process often results in high predictive accuracy. It is especially useful when you need a model that can capture complex patterns in the data while still being interpretable and customizable.

In [12]:
ps = RandomParameterSampling({
    "learning_rate": choice(0.001,0.01,0.02, 0.05, 0.1, 0.25 ,0.75 , 1.0),
    "n_estimators": choice(1,3 , 5,7, 10,15,20, 25)
})

import os

policy = BanditPolicy(
    evaluation_interval=5,
    slack_factor=0.3)

if "capstone-file-hyper" not in os.listdir():
    os.mkdir("./capstone-file-hyper")


sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')

src = ScriptRunConfig(
    source_directory="./",
    script="train_dataset.py",
    compute_target=compute_target,
    environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=17,
    max_concurrent_runs=5)

In [15]:
hyperdrive_run = experiment.submit(hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e
Web View: https://ml.azure.com/runs/HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-269450/workspaces/quick-starts-ws-269450&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-03T17:07:48.3327589Z][GENERATOR][DEBUG]Sampled 5 jobs from search space 
[2024-11-03T17:07:48.6025525Z][SCHEDULER][INFO]Scheduling job, id='HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_0' 
[2024-11-03T17:07:48.6588882Z][SCHEDULER][INFO]Scheduling job, id='HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_1' 
[2024-11-03T17:07:48.7480097Z][SCHEDULER][INFO]Scheduling job, id='HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_4' 
[2024-11-03T17:07:48.7003165Z][SCHEDULER][INFO]Scheduling job, id='HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_2' 
[2024-11-03T17:07:48.7515466Z][SCHEDULER][INFO]Scheduling job, id='HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_3' 
[2024-11-03T17:07:49.4740505Z]

{'runId': 'HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e',
 'target': 'training-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-11-03T17:07:46.961419Z',
 'endTimeUtc': '2024-11-03T17:12:50.229424Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6b7ecd6d-8557-4686-b465-1a3de05f15b1',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_9',
  'score': '0.8',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',
   

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [17]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print(f"Best Model Run ID: {best_run.id}\n")
print(f"Best Model Metrics: {best_run_metrics}\n")
print(f"Best Model Accuracy: {best_run_metrics['Accuracy']}\n")

Best Model Run ID: HD_f012b6b0-6f40-408a-8302-6bd4c2760e0e_9

Best Model Metrics: {'Learning Rate:': 0.75, 'Number Estimators:': 15, 'Accuracy': 0.8}

Best Model Accuracy: 0.8



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
model = best_run.register_model(model_path="trained_model.pkl",model_name="best_model_gradient_boosting",
                               properties={
                                   "Accuracy": best_run_metrics["Accuracy"],
                                   "Number Estimators": best_run_metrics["Number Estimators:"],
                                   "Learning Rate": best_run_metrics["Learning Rate:"]
                               })

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

